##Sistema de Recomendação baseado em conteúdo utilizando a similaridade do cosseno(por conta de uma matriz esparsa) e o tf-idf para representar a relevância entre as categorias  

fonte : https://www.kaggle.com/datasets/karkavelrajaj/amazon-sales-dataset

In [42]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics.pairwise import cosine_similarity

In [43]:
df = pd.read_csv("/content/amazon.csv")
df

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",64%,4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,₹199,₹349,43%,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,₹199,"₹1,899",90%,3.9,"7,928",【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","Kunal,Himanshu,viswanath,sai niharka,saqib mal...","R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Sounce-iPhone-Charging-C...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,₹329,₹699,53%,4.2,"94,363",The boAt Deuce USB 300 2 in 1 cable is compati...,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...","Omkar dhale,JD,HEMALATHA,Ajwadh a.,amar singh ...","R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou...",https://m.media-amazon.com/images/I/41V5FtEWPk...,https://www.amazon.in/Deuce-300-Resistant-Tang...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,₹154,₹399,61%,4.2,"16,905",[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...","rahuls6099,Swasat Borah,Ajay Wadke,Pranali,RVK...","R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...","As good as original,Decent,Good one for second...","Bought this instead of original apple, does th...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Portronics-Konnect-POR-1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,B08L7J3T31,Noir Aqua - 5pcs PP Spun Filter + 1 Spanner | ...,Home&Kitchen|Kitchen&HomeAppliances|WaterPurif...,₹379,₹919,59%,4,"1,090",SUPREME QUALITY 90 GRAM 3 LAYER THIK PP SPUN F...,"AHITFY6AHALOFOHOZEOC6XBP4FEA,AFRABBODZJZQB6Z4U...","Prabha ds,Raghuram bk,Real Deal,Amazon Custome...","R3G3XFHPBFF0E8,R3C0BZCD32EIGW,R2EBVBCN9QPD9R,R...","Received the product without spanner,Excellent...","I received product without spanner,Excellent p...",https://m.media-amazon.com/images/I/41fDdRtjfx...,https://www.amazon.in/Noir-Aqua-Spanner-Purifi...
1461,B01M6453MB,Prestige Delight PRWO Electric Rice Cooker (1 ...,Home&Kitchen|Kitchen&HomeAppliances|SmallKitch...,"₹2,280","₹3,045",25%,4.1,"4,118","230 Volts, 400 watts, 1 Year","AFG5FM3NEMOL6BNFRV2NK5FNJCHQ,AGEINTRN6Z563RMLH...","Manu Bhai,Naveenpittu,Evatira Sangma,JAGANNADH...","R3DDL2UPKQ2CK9,R2SYYU1OATVIU5,R1VM993161IYRW,R...","ok,everything was good couldn't return bcoz I ...","ok,got everything as mentioned but the me

In [44]:
# Separando as palavras das categorias e observando sua contagem

pd.set_option('display.max_rows', 500)
# category_popularity = (df.category.str.split("|").value_counts().sort_values(ascending=False)) 
category_popularity = (df.category.str.split("|").explode().value_counts().sort_values(ascending=False)) 
# .explode() separa os valores das listas e os trata como indexes
category_popularity

Electronics                                526
Computers&Accessories                      453
Home&Kitchen                               448
Accessories&Peripherals                    381
Kitchen&HomeAppliances                     308
Cables                                     267
Cables&Accessories                         240
USBCables                                  233
SmallKitchenAppliances                     181
HomeTheater,TV&Video                       162
Mobiles&Accessories                        161
Heating,Cooling&AirQuality                 116
Accessories                                111
MobileAccessories                           84
Vacuum,Cleaning&Ironing                     82
Smartphones&BasicMobiles                    77
SmartWatches                                76
WearableTechnology                          76
Televisions                                 69
Smartphones                                 68
Headphones,Earbuds&Accessories              66
SmartTelevisi

In [45]:
# Existem 317 categorias 
category_name = (df.category.str.split("|").explode().value_counts().sort_values(ascending=False).index)
len(category_name)

317

In [46]:
df.dtypes

product_id             object
product_name           object
category               object
discounted_price       object
actual_price           object
discount_percentage    object
rating                 object
rating_count           object
about_product          object
user_id                object
user_name              object
review_id              object
review_title           object
review_content         object
img_link               object
product_link           object
dtype: object

In [47]:
df.isnull().sum()

product_id             0
product_name           0
category               0
discounted_price       0
actual_price           0
discount_percentage    0
rating                 0
rating_count           2
about_product          0
user_id                0
user_name              0
review_id              0
review_title           0
review_content         0
img_link               0
product_link           0
dtype: int64

In [48]:
df.dropna(inplace=True)

In [49]:
df.isnull().sum()

product_id             0
product_name           0
category               0
discounted_price       0
actual_price           0
discount_percentage    0
rating                 0
rating_count           0
about_product          0
user_id                0
user_name              0
review_id              0
review_title           0
review_content         0
img_link               0
product_link           0
dtype: int64

# If-Idf
## PESO utilizado para mensurar a relevância de uma palavra em um texto

In [50]:
tfidf = TfidfVectorizer(stop_words="english")

matriz_tfidf = tfidf.fit_transform(df["category"])
matriz_tfidf.shape

(1463, 367)

In [51]:
print(tfidf.get_feature_names_out())

['3dglasses' 'accessories' 'adapters' 'airconditioners' 'airfryers'
 'airpurifiers' 'airquality' 'amplifiers' 'arts' 'audio'
 'automobileaccessories' 'automobilechargers' 'avreceivers'
 'backgroundsupports' 'bags' 'basic' 'basiccases' 'basicmobiles'
 'batteries' 'batterychargers' 'bedstand' 'bluetoothadapters'
 'bluetoothspeakers' 'bottledink' 'business' 'cableconnectionprotectors'
 'cables' 'caddies' 'calculators' 'cameraprivacycovers' 'cameras'
 'canistervacuums' 'car' 'caraccessories' 'cartridges' 'cases'
 'ceilingfans' 'ceilingmounts' 'chargers' 'chippers' 'choppers' 'cleaners'
 'cleaning' 'cleaningkits' 'coffee' 'coffeegrinders'
 'coffeemakeraccessories' 'coffeepresses' 'coldpressjuicers'
 'colouredpaper' 'colouringpens' 'completetripodunits' 'components'
 'compositionnotebooks' 'computers' 'condenser' 'cooling' 'coolingpads'
 'copy' 'cordmanagement' 'covers' 'cradles' 'craftmaterials' 'crafts'
 'datacards' 'datastorage' 'deepfatfryers' 'deskmounts' 'diaries'
 'digitalbathroomscal

In [52]:
matriz_tfidf.toarray()

array([[0.        , 0.54992372, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.54992372, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.54992372, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [53]:
pd.DataFrame(matriz_tfidf.toarray(),columns=tfidf.get_feature_names_out(),index=df.product_name)

,3dglasses,accessories,adapters,airconditioners,airfryers,airpurifiers,airquality,amplifiers,arts,audio,...,weighingscales,wet,wetgrinders,windowcleaners,wireboundnotebooks,wirelessusbadapters,woodenpencils,writingpads,writingsupplies,yogurtmakers
product_name,,,,,,,,,,,,,,,,,,,,,
"Wayona Nylon Braided USB to Lightning Fast Charging and Data Sync Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini (3 FT Pack of 1, Grey)",0.0,0.549924,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Ambrane Unbreakable 60W / 3A Fast Charging 1.5m Braided Type C Cable for Smartphones, Tablets, Laptops & other Type C devices, PD Technology, 480Mbps Data Sync, Quick Charge 3.0 (RCT15A, Black)",0.0,0.549924,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Sounce Fast Phone Charging Cable & Data Sync USB Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini & iOS Devices",0.0,0.549924,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"boAt Deuce USB 300 2 in 1 Type-C & Micro USB Stress Resistant, Tangle-Free, Sturdy Cable with 3A Fast Charging & 480mbps Data Transmission, 10000+ Bends Lifespan and Extended 1.5m Length(Martian Red)",0.0,0.549924,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Portronics Konnect L 1.2M Fast Charging 3A 8 Pin USB Cable with Charge & Sync Function for iPhone, iPad (Grey)",0.0,0.549924,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Noir Aqua - 5pcs PP Spun Filter + 1 Spanner | for All Types of RO Water purifiers (5 Piece, White, 10 Inch, 5 Micron) - RO Spun Filter Cartridge Sponge Replacement Water Filter Candle",0.0,0.157872,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Prestige Delight PRWO Electric Rice Cooker (1 L, White)",0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Bajaj Majesty RX10 2000 Watts Heat Convector Room Heater (White, ISI Approved)",0.0,0.000000,0.0,0.0,0.0,0.0,0.324336,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Similaridade do Cosseno

In [54]:
cosine_sim = cosine_similarity(matriz_tfidf)

cosine_sim_df = pd.DataFrame(cosine_sim,index=df["product_name"],columns=df["product_name"])
cosine_sim_df.round(2)

product_name,"Wayona Nylon Braided USB to Lightning Fast Charging and Data Sync Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini (3 FT Pack of 1, Grey)","Ambrane Unbreakable 60W / 3A Fast Charging 1.5m Braided Type C Cable for Smartphones, Tablets, Laptops & other Type C devices, PD Technology, 480Mbps Data Sync, Quick Charge 3.0 (RCT15A, Black)","Sounce Fast Phone Charging Cable & Data Sync USB Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini & iOS Devices","boAt Deuce USB 300 2 in 1 Type-C & Micro USB Stress Resistant, Tangle-Free, Sturdy Cable with 3A Fast Charging & 480mbps Data Transmission, 10000+ Bends Lifespan and Extended 1.5m Length(Martian Red)","Portronics Konnect L 1.2M Fast Charging 3A 8 Pin USB Cable with Charge & Sync Function for iPhone, iPad (Grey)","pTron Solero TB301 3A Type-C Data and Fast Charging Cable, Made in India, 480Mbps Data Sync, Strong and Durable 1.5-Meter Nylon Braided USB Cable for Type-C Devices for Charging Adapter (Black)","boAt Micro USB 55 Tangle-free, Sturdy Micro USB Cable with 3A Fast Charging & 480mbps Data Transmission (Black)",MI Usb Type-C Cable Smartphone (Black),"TP-Link USB WiFi Adapter for PC(TL-WN725N), N150 Wireless Network Adapter for Desktop - Nano Size WiFi Dongle Compatible with Windows 11/10/7/8/8.1/XP/ Mac OS 10.9-10.15 Linux Kernel 2.6.18-4.4.3","Ambrane Unbreakable 60W / 3A Fast Charging 1.5m Braided Micro USB Cable for Smartphones, Tablets, Laptops & Other Micro USB Devices, 480Mbps Data Sync, Quick Charge 3.0 (RCM15, Black)",...,"USHA 1212 PTC with Adjustable Thermostat Fan Heater (Black/Brown, 1500-Watts).","4 in 1 Handheld Electric Vegetable Cutter Set,Wireless Food Processor Electric Food Chopper for Garlic Chili Pepper Onion Ginger Celery Meat with Brush",Philips HD9306/06 1.5-Litre Electric Kettle (Multicolor),"Libra Room Heater for Home, Room Heaters Home for Winter, Electric Heater with 2000 Watts Power as per IS Specification for Small to Medium Rooms - FH12 (Grey)",NGI Store 2 Pieces Pet Hair Removers for Your Laundry Catcher Lint Remover for Washing Machine Lint Remover Reusable Portable Silica Gel Clothes Washer Dryer Floating Ball,"Noir Aqua - 5pcs PP Spun Filter + 1 Spanner | for All Types of RO Water purifiers (5 Piece, White, 10 Inch, 5 Micron) - RO Spun Filter Cartridge Sponge Replacement Water Filter Candle","Prestige Delight PRWO Electric Rice Cooker (1 L, White)","Bajaj Majesty RX10 2000 Watts Heat Convector Room Heater (White, ISI Approved)",Havells Ventil Air DSP 230mm Exhaust Fan (Pista Green),Borosil Jumbo 1000-Watt Grill Sandwich Maker (Black)
product_name,,,,,,,,,,,,,,,,,,,,,
"Wayona Nylon Braided USB to Lightning Fast Charging and Data Sync Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini (3 FT Pack of 1, Grey)",1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.15,1.00,...,0.00,0.00,0.00,0.00,0.07,0.09,0.00,0.00,0.00,0.00
"Ambrane Unbreakable 60W / 3A Fast Charging 1.5m Braided Type C Cable for Smartphones, Tablets, Laptops & other Type C devices, PD Technology, 480Mbps Data Sync, Quick Charge 3.0 (RCT15A, Black)",1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.15,1.00,...,0.00,0.00,0.00,0.00,0.07,0.09,0.00,0.00,0.00,0.00
"Sounce Fast Phone Charging Cable & Data Sync USB Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini & iOS Devices",1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.15,1.00,...,0.00,0.00,0.00,0.00,0.07,0.09,0.00,0.00,0.00,0.00
"boAt Deuce USB 300 2 in 1 Type-C & Micro USB Stress Resistant, Tangle-Free, Sturdy Cable with 3A Fast Charging & 480mbps Data Transmission, 10000+ Bends Lifespan and Extended 1.5m Length(Martian Red)",1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.15,1.00,...,0.00,0.00,0.00,0.00,0.07,0.09,0.00,0.00,0.00,0.00
"Portronics Konnect L 1.2M Fast Charging 3A 8 Pin USB Cable with Charge & Sync Function for iPhone, iPad (Grey)",1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.15,1.00,...,0.00,0.00,0.00,0.00,0.07,0.09,0.00,0.00,0.00,0.00
...,...

In [55]:
cosine_sim_df.shape

(1463, 1463)

# Fazendo a recomendação do produto 

In [56]:
import random
r = random.randint(0, df.shape[0])
df.iloc[r]

product_id                                                    B0B1F6GQPS
product_name           Boult Audio FXCharge with ENC, 32H Playtime, 5...
category               Electronics|Headphones,Earbuds&Accessories|Hea...
discounted_price                                                    ₹999
actual_price                                                      ₹4,499
discount_percentage                                                  78%
rating                                                               3.8
rating_count                                                       3,390
about_product          Environmental Noise Cancellation: Equipped wit...
user_id                AGCWHOWHOTWSN4J2TFAXUEZZUBXQ,AF7COMJXY3YJUCEUE...
user_name              V.W.,Nitin desai,Amazon Customer,Kaushallya De...
review_id              R2888CE3TDHQMW,R5OOQZ5ILIG7E,R3CCDJLE61ON18,R1...
review_title           Noise cancellation is just a hype,Okay,Sound,g...
review_content         Good :Sound quality - surpri

In [57]:
p_name = df.iloc[r].product_name
p_name

'Boult Audio FXCharge with ENC, 32H Playtime, 5min=7H Type C Fast Charging, Zen ENC, 14.2 mm BoomX Rich Bass, IPX5, Bluetooth Wireless in Ear Earphones Neckband with mic (Black)'

In [59]:
def category_recomendation(alvo,cosine_sim_df,df,items):
  try:
    # Achando os 10 produtos mais similares
    closest = cosine_sim_df[alvo].sort_values(ascending=False).head(10)

    # Transformando em um Dataframe os produtos mais similares
    closest = pd.DataFrame(dict(product_name=closest.index))

    # Pega os dados do dataframe principal, se os nomes dos produtos estiverem no dataframe dos mais similares e também
    # no dataframe principal 
    recomendation = df.loc[df['product_name'].isin(closest['product_name'])].reset_index(drop=True)

    # DataFrame final com as colunas nome do produto e categoria
    recomendation = recomendation[["product_name","category"]]
  except:
    print("poucos dados relacionados ao objeto alvo, porfavor escolha outro para o sistema recomendar")
  
  return recomendation


In [60]:
Recomendation = category_recomendation(p_name,cosine_sim_df,df,df[["product_name","category"]])
Recomendation

,product_name,category
0,"JBL C100SI Wired In Ear Headphones with Mic, J...","Electronics|Headphones,Earbuds&Accessories|Hea..."
1,boAt Bassheads 100 in Ear Wired Earphones with...,"Electronics|Headphones,Earbuds&Accessories|Hea..."
2,ZEBRONICS Zeb-Bro in Ear Wired Earphones with ...,"Electronics|Headphones,Earbuds&Accessories|Hea..."
3,"JBL C100SI Wired In Ear Headphones with Mic, J...","Electronics|Headphones,Earbuds&Accessories|Hea..."
4,boAt Rockerz 450 Bluetooth On Ear Headphones w...,"Electronics|Headphones,Earbuds&Accessories|Hea..."
5,"JBL C50HI, Wired in Ear Headphones with Mic, O...","Electronics|Headphones,Earbuds&Accessories|Hea..."
6,Boult Audio BassBuds X1 in-Ear Wired Earphones...,"Electronics|Headphones,Earbuds&Accessories|Hea..."
7,Logitech H111 Wired On Ear Headphones With Mic...,"Electronics|Headphones,Earbuds&Accessories|Hea..."
8,"JBL Tune 215BT, 16 Hrs Playtime with Quick Cha...","Electronics|Headphones,Earbuds&Accessories|Hea..."
9,"Boult Audio Truebuds with 30H Playtime, IPX7 W...","Electronics|Headphones,Earbuds&Accessories|Hea..."
